#### Char-level Language Model - Part 3

* Char-Transformer: A char-level transformer based language model trained on a toy dataset of Shakespeare's work to predict Shakespeasre like language.

In [5]:
import torch

In [6]:
# The Tiny Shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-02-06 23:19:24--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt.1'

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.08s   

2023-02-06 23:19:24 (13.0 MB/s) - 'input.txt.1' saved [1115394/1115394]



In [7]:
with open("input.txt", 'r', encoding='utf-8') as file:
    input_text = file.read()

In [8]:
print(f"Length of input text: {len(input_text)}")
print(input_text[:1000])

Length of input text: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread

In [9]:
# create a character vocabulary
char_vocab = sorted(set(input_text))
vocab_size = len(char_vocab)
print(f"{vocab_size=} and characters in vocab: {''.join(char_vocab)}")
# In our vocab, first character is new line character '\n' and second character is space ' '.

vocab_size=65 and characters in vocab: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


#### Tokenization: Text to integer mapping

We will be using character to integer tokenizer.

Other possible tokenizsers, Sub-word tokenizers:
* [OpenAI's tiktoken](https://github.com/openai/tiktoken)
* [Google's SentencePiece](https://pypi.org/project/sentencepiece/)

In [10]:
# character to index mapping
chartoi = {}
# index to character mapping
itochar = {}
for i, c in enumerate(char_vocab):
    chartoi[c] = i
    itochar[i] = c
print(f"{chartoi=}")
print(f"{itochar=}")

encode = lambda char_seq: [chartoi[char] for char in char_seq]
decode = lambda idx_seq: "".join([itochar[idx] for idx in idx_seq])

chartoi={'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
itochar={0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 4

In [11]:
print(f"Encode: {input_text[:30]}")
print(f"Encoded sentence:{encode(input_text[:30])}\nDecoded sentence:{decode(encode(input_text[:30]))}")


Encode: First Citizen:
Before we proce
Encoded sentence:[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43]
Decoded sentence:First Citizen:
Before we proce


In [22]:
# encode entire text
data = torch.tensor(encode(input_text), dtype=torch.long)
data[:20]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56])

Train/Val split

In [36]:
n_train = int(0.9*len(data))
train_data = data[:n_train]
val_data = data[n_train:]
print(f"n_train:{n_train} and n_val: {val_data.shape[0]}")

n_train:1003854 and n_val: 111540


Create batch

In [39]:
# context length/block size/chunk size
block_size = 8
batch_size = 4
def build_batch(split):
    data = train_data if split == "train" else val_data
    start_idxs = torch.randint(0, len(data)-block_size, size=(batch_size,))
    #input 
    X = torch.stack([data[idx:idx+block_size] for idx in start_idxs])
    #target (input shift right by one)
    Y = torch.stack([data[idx+1:idx+block_size+1] for idx in start_idxs])
    return X, Y

In [40]:
X, Y = build_batch(input_text)

Each input sequence of length `block_size` has `block_size` number of inputs packed into it.

In [41]:
for b in range(1): # batch dimension
    print(f"Input: {X[b]} and target: {Y[b]}")
    for t in range(block_size): # time dimension
        context = X[b, :t+1]
        target = Y[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
print(f"Input to the transformer:\n{X}")

Input: tensor([47, 52, 58, 43, 42,  1, 61, 47]) and target: tensor([52, 58, 43, 42,  1, 61, 47, 58])
when input is [47] the target: 52
when input is [47, 52] the target: 58
when input is [47, 52, 58] the target: 43
when input is [47, 52, 58, 43] the target: 42
when input is [47, 52, 58, 43, 42] the target: 1
when input is [47, 52, 58, 43, 42, 1] the target: 61
when input is [47, 52, 58, 43, 42, 1, 61] the target: 47
when input is [47, 52, 58, 43, 42, 1, 61, 47] the target: 58
Input to the transformer:
tensor([[47, 52, 58, 43, 42,  1, 61, 47],
        [57, 39, 61,  1, 63, 53, 59,  1],
        [ 6,  1, 39, 52, 42,  1, 43, 60],
        [ 1, 50, 53, 60, 43,  1, 40, 43]])


#### Baseline
Bigram neural network model using pytorch embedding layer